In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

sns.set(rc={'figure.figsize': (30, 15)}, font_scale=1.5)

df = pd.read_csv('ECH_2019.csv')

df.info()

In [ ]:
df.sample(5)

In [ ]:
no_data = {0: 'No hay dato', '0': 'No hay dato'}

mask = df.columns.difference(['edad', 'sueldo', 'hijos'])

df[mask] = df[mask].replace(no_data)

df.sample(5)

In [ ]:
df['estado_laboral'].value_counts()

Seleccionamos personas con edad suficiente para trabajar

In [ ]:
df = df[~df['estado_laboral'].str.contains('Menores de 14 años')]

Codificamos los valores de parentezco

In [ ]:
df['parentesco_num'] = df['parentesco'].astype('category').cat.codes

df[['parentesco_num', 'parentesco']].value_counts()

Creamos una columna que cuenta la cantidad de hijos por grupo familiar

In [ ]:
df['hijos'] = (
    df.groupby('id_hogar')['id_hogar'].transform('count') - 1
)

### AJUSTANDO DATOS PARA ENTRENAR EL MODELO

Primero encodificamos las variables categóricas

In [ ]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

mask = df.columns.difference(['edad', 'sueldo', 'hijos'])

df[mask] = df[mask].apply(le.fit_transform)

df.head(5)

Segundo escalamos todos los atributos

In [ ]:
from sklearn.preprocessing import StandardScaler

df = pd.DataFrame(StandardScaler().fit_transform(df), columns=df.columns)

df

### Creamos un mapa de correlación para investigar la relación entre las variables

In [ ]:
corr = df.corr()
sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values, annot=True)

### DEFINIMOS LAS VARIABLES DEPENDIENTES E INDEPENDIENTES

In [ ]:
# Feautures
X = df.drop('estado_laboral', axis=1)
y = df['estado_laboral']

print('X shape', X.shape, '\nY shape', y.shape)

In [ ]:
np.unique(y)

### Dividimos los datos en un 30% de test y 70% de entrenamiento

In [ ]:
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42) 

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### DEFINIMOS ALGORITMO DE REFRESIÓN LOGISTICA Y VALOR DE CROSS VALIDATION PARA CADA FOLD

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

log_reg = LogisticRegression(max_iter=20000)

clf = log_reg.fit(X_train.astype('int'), y_train.astype('int'))

scores = cross_val_score(clf, X.astype('int'), y.astype('int'), cv=10)

scores

Promedio de scores en el Cross Validation

In [ ]:
scores.mean()

Scores de cada clasificación

In [ ]:
clf.score(X_test, y_test)

In [ ]:
clf.score(X_train, y_train)

### DEFINIMOS UNA PREDICCIÓN Y LA APLICAMOS SOBRE EL MODELO

In [ ]:
prediction = np.array([10000, 20000, 5000, 80000, 3000, 100,30000]).reshape(-1,1)
log_reg.predict(prediction)

### INVESTIGAMOS LA EFECTIVIDAD DEL MODELO SOBRE LOS DATOS

In [ ]:
from sklearn.metrics import accuracy_score

#Prediccion en Train
y_train_pred = log_reg.predict(X_train)

#Prediccion en Test
y_test_pred = log_reg.predict(y_test.reshape(-1, 1))

#Calculo el accuracy en Train
train_accuracy = accuracy_score(y_train, y_train_pred)

#Calculo el accuracy en Test
test_accuracy = accuracy_score(y_test, y_test_pred)

print('% de aciertos sobre el set de entrenamiento:', train_accuracy)
print('% de aciertos sobre el set de evaluación:',test_accuracy)

Determinamos la probabilidad de la predicción

In [ ]:
probabilidades_prediccion = log_reg.predict_proba(prediction)
probabilidades_prediccion[:,1]